In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import shutil
import cv2
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.utils import to_categorical
from itertools import cycle
from skimage.feature import local_binary_pattern
from skimage import img_as_ubyte
from tensorflow.keras import backend as K

strategy = tf.distribute.MirroredStrategy()

Load image function****

In [ ]:
# Load image and mask function
def load_images_and_masks(image_folder, mask_folder, image_size=(256, 256)):
    images = []
    masks = []
    labels = []
    class_names = sorted(os.listdir(image_folder))
    label_to_index = {name: index for index, name in enumerate(class_names)}
    
    for label in class_names:
        class_folder_images = os.path.join(image_folder, label)
        class_folder_masks = os.path.join(mask_folder, label)
        if os.path.isdir(class_folder_images):
            for filename in os.listdir(class_folder_images):
                img_path = os.path.join(class_folder_images, filename)
                mask_path = os.path.join(class_folder_masks, filename) if os.path.isfile(os.path.join(class_folder_masks, filename)) else None
                
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, image_size)
                    images.append(img)
                    
                    if mask_path and os.path.isfile(mask_path):
                        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                        mask = cv2.resize(mask, image_size)
                        masks.append(mask)
                    
                    labels.append(label_to_index[label])
    
    return np.array(images), np.array(masks), np.array(labels)

Function to increase contrast****

In [ ]:
# Function to increase contrast
def increase_contrast(image, alpha=1.7):
    return cv2.convertScaleAbs(image, alpha=alpha)

def apply_mask(image, mask, threshold=128):
    image = np.uint8(image)
    mask = np.uint8(mask)
    _, binary_mask = cv2.threshold(mask, threshold, 255, cv2.THRESH_BINARY)
    masked_image = cv2.bitwise_and(image, image, mask=binary_mask)
    return masked_image

# Preprocess images by increasing contrast and applying mask
def preprocess_images(images, masks):
    grayscale_images = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images])
    contrasted_images = np.array([increase_contrast(img) for img in grayscale_images])
    masked_images = np.array([apply_mask(img, mask) for img, mask in zip(contrasted_images, masks)])
    return masked_images

# Modify labels to match the output shape of the U-Net model
def preprocess_labels(labels, height, width, num_classes):
    masks = np.zeros((len(labels), height, width, num_classes), dtype=np.uint8)
    for i, label in enumerate(labels):
        masks[i, :, :, label] = 1
    return masks

Load the train and test data****

In [ ]:
# Load the train and test data
train_dir = '/kaggle/input/brain-tumor-mri-dataset/Training'
test_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing'

train_images, train_masks, train_labels = load_images_and_masks(train_dir, train_dir)
test_images, test_masks, test_labels = load_images_and_masks(test_dir, test_dir)

# Preprocess images
train_images_processed = preprocess_images(train_images, train_masks)
test_images_processed = preprocess_images(test_images, test_masks)

# Convert labels to categorical (one-hot encoding)
num_classes = len(set(train_labels))  # Number of classes
train_labels_processed = preprocess_labels(train_labels, 256, 256, num_classes)
test_labels_processed = preprocess_labels(test_labels, 256, 256, num_classes)

Display Images

In [ ]:
# Display images
def display_images(original_images, processed_images, masks, labels, class_names, num_examples=2):
    unique_labels = sorted(set(labels))
    fig, axs = plt.subplots(len(unique_labels), num_examples * 2, figsize=(15, 12))
    
    for idx, label in enumerate(unique_labels):
        indices = [i for i, l in enumerate(labels) if l == label][:num_examples]
        for i, index in enumerate(indices):
            axs[idx, i * 2].imshow(original_images[index], cmap='gray')  # Original image
            axs[idx, i * 2].set_title(f'Original - {class_names[label]}')
            axs[idx, i * 2].axis('off')
            
            axs[idx, i * 2 + 1].imshow(processed_images[index], cmap='gray')  # Processed image
            axs[idx, i * 2 + 1].set_title(f'Processed - {class_names[label]}')
            axs[idx, i * 2 + 1].axis('off')
    
    plt.tight_layout()
    plt.show()

class_names = sorted(os.listdir(train_dir))
display_images(test_images, test_images_processed, test_masks, list(test_labels), class_names)

Defining the model****

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

def vgg16_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    # Load VGG16 without pre-trained weights
    base_model = VGG16(weights=None, include_top=False, input_tensor=inputs)
    
    # Make sure all layers in the base model are trainable
    base_model.trainable = True
    
    # Add custom layers on top of the base model
    x = base_model.output
    # Add additional convolutional layers
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Flatten()(x)  # Flatten the output to a 1D vector
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # Create the model
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

input_shape = (256, 256, 1)  # Use grayscale input (1 channel)
model = vgg16_model(input_shape, num_classes)
model.summary()

****Train the dataset

In [ ]:
# Convert labels to one-hot encoding
def one_hot_encode(labels, num_classes):
    return tf.keras.utils.to_categorical(labels, num_classes=num_classes)

# Example: Convert labels
train_labels_one_hot = one_hot_encode(train_labels, num_classes)
test_labels_one_hot = one_hot_encode(test_labels, num_classes)



from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8, verbose=1)

with tf.distribute.MirroredStrategy().scope():
    model = vgg16_model(input_shape, num_classes)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

# Model checkpoints and early stopping
best_model_path = 'best_model_weights.keras'

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    best_model_path,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)
# Train the model
history = model.fit(
    train_images_processed,  # Processed grayscale images
    train_labels_one_hot,    # One-hot encoded labels
    epochs=100,
    batch_size=32,
    validation_data=(test_images_processed, test_labels_one_hot),
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images_processed, test_labels_one_hot)
print(f"Test Accuracy: {test_accuracy:.5f}")

Graphs****

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import seaborn as sns
# from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, label_binarize

# Plot accuracy and loss curves
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curve')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')

plt.show()

# Predictions
predictions = model.predict(test_images_processed)
predictions_classes = np.argmax(predictions, axis=-1).flatten()

# Ensure test_labels_one_hot is used correctly
test_labels_one_hot = np.array(test_labels_one_hot)
test_labels_classes = np.argmax(test_labels_one_hot, axis=-1).flatten()

# Classification report
print("Classification Report:")
print(classification_report(test_labels_classes, predictions_classes, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(test_labels_classes, predictions_classes)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# ROC Curve
binarized_labels = label_binarize(test_labels_classes, classes=range(num_classes))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(binarized_labels[:, i], predictions[:, i])
    plt.plot(fpr, tpr, label=f'Class {class_names[i]} (area = {auc(fpr, tpr):.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()


In [ ]:
model.save("model.h5")